<a href="https://colab.research.google.com/github/psbruno/fairness/blob/main/Modelos_padr%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
ad = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/ADMISSIONS.csv.gz")
diag = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv.gz")
d_diag = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/D_ICD_DIAGNOSES.csv.gz")
d_le = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/D_LABITEMS.csv.gz")
le = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/LABEVENTS.csv.gz")
pat = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/PATIENTS.csv.gz")
proc = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/PROCEDURES_ICD.csv.gz")

In [4]:
ad = pd.read_csv("/content/physionet.org/files/mimiciii/1.4/ADMISSIONS.csv.gz")
ad.loc[ad['DIAGNOSIS'] == 'CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT /SDA','DIAGNOSIS'] = 'CORONARY ARTERY DISEASE'
ad.loc[ad['DIAGNOSIS'] == 'CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS GRAFT/SDA','DIAGNOSIS'] = 'CORONARY ARTERY DISEASE'
ad.loc[ad['DIAGNOSIS'] == 'CORONARY ARTERY DISEASE\CATH','DIAGNOSIS'] = 'CORONARY ARTERY DISEASE'

In [5]:
import itertools
examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

examsList = [str(i) for i in examsList]
def makeDF(examsSubset,data):
  if data.shape[0] == 0:
    return
  examsSubset['SUBJECT_ID'].append(data.loc[0,'SUBJECT_ID'])
  examsSubset['HADM_ID'].append(data.loc[0,'HADM_ID'])
  for i in examsList:
    i = int(i)
    if i in list(data['ITEMID']):
       examsSubset[str(i)].append(sum(list(data[data['ITEMID']==i]['VALUENUM']))/len(list(data[data['ITEMID']==i]['VALUENUM'])))
    else:
      examsSubset[str(i)].append(None)
def buildTests(columns):

  patientsHADM = list(ad[ad['DIAGNOSIS'].isin(columns)].reset_index()['HADM_ID'].drop_duplicates())
  ad[ad['HADM_ID'].isin(patientsHADM)].groupby("DIAGNOSIS").sum()
  examsSubset = {str(i):[] for i in list(examsList)}
  examsSubset['SUBJECT_ID'] = []
  examsSubset['HADM_ID'] = []
  patientsTarget = le[le['HADM_ID'].isin(patientsHADM)]
  patientsTarget = patientsTarget.reset_index().drop(columns='index')
  counter = 0
  for i in patientsHADM:
    makeDF(examsSubset,patientsTarget[patientsTarget['HADM_ID']==i].reset_index())
    counter+=1
    if counter % 500 == 0:
      print(counter)
  df = pd.DataFrame(examsSubset)
  for i in examsList:
    if i in list(df.columns):
      df[i] = df[i].fillna(df[i].mean())
      df[i] = (df[i] - df[i].mean())/df[i].std()
  return df
dfs = []
count = 0
combList = []
for testDisease in ['DIABETIC KETOACIDOSIS']:
  result = buildTests(['CORONARY ARTERY DISEASE','CONGESTIVE HEART FAILURE',testDisease])
  dfs.append([pd.merge(result, ad, on='HADM_ID', how='inner')])
  count +=1
  print("\n\n\n\n\n")
  print(count)
  print("\n\n\n\n\n")

for i in range(len(dfs)):
  dfs[i][0]['Reduced_ETHNICITY'] = dfs[i][0]['ETHNICITY']
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC OR LATINO','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='BLACK/AFRICAN AMERICAN','Reduced_ETHNICITY'] = 'BLACK'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='UNKNOWN/NOT SPECIFIED','Reduced_ETHNICITY']= 'OTHER'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - CHINESE','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='WHITE - EASTERN EUROPEAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='PATIENT DECLINED TO ANSWER','Reduced_ETHNICITY'] = 'OTHER'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='BLACK/AFRICAN','Reduced_ETHNICITY'] = 'BLACK'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - CAMBODIAN','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='BLACK/HAITIAN','Reduced_ETHNICITY'] = 'BLACK'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='WHITE - RUSSIAN','Reduced_ETHNICITY'] = 'WHITE'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='UNABLE TO OBTAIN','Reduced_ETHNICITY'] = 'OTHER'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='WHITE - BRAZILIAN','Reduced_ETHNICITY'] = 'WHITE'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='BLACK/CAPE VERDEAN','Reduced_ETHNICITY'] = 'BLACK'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - VIETNAMESE','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='MULTI RACE ETHNICITY','Reduced_ETHNICITY'] = 'OTHER'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='WHITE - OTHER EUROPEAN','Reduced_ETHNICITY'] = 'WHITE'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - ASIAN INDIAN','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - MEXICAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - CUBAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - KOREAN','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - GUATEMALAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - PUERTO RICAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - SALVADORAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - DOMINICAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='SOUTH AMERICAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='CARIBBEAN ISLAND','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - FILIPINO','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - COLOMBIAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - JAPANESE - COLOMBIAN','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - THAI','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='HISPANIC/LATINO - HONDURAN','Reduced_ETHNICITY'] = 'LATINO'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - OTHER','Reduced_ETHNICITY'] = 'ASIAN'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER','Reduced_ETHNICITY'] = 'OTHER'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='AMERICAN INDIAN/ALASKA NATIVE','Reduced_ETHNICITY'] = 'OTHER'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='PORTUGUESE','Reduced_ETHNICITY'] = 'OTHER'
  dfs[i][0].loc[dfs[i][0]['ETHNICITY']=='ASIAN - JAPANESE','Reduced_ETHNICITY'] = 'ASIAN'


  dfs[i][0].loc[dfs[i][0]['MARITAL_STATUS']=='LIFE PARTNER','MARITAL_STATUS'] = 'UNKNOWN (DEFAULT)'
  dfs[i][0].loc[dfs[i][0]['MARITAL_STATUS'].isna(),'MARITAL_STATUS'] = 'UNKNOWN (DEFAULT)'
  dfs[i][0].loc[dfs[i][0]['MARITAL_STATUS']== 'UNKNOWN (DEFAULT)','MARITAL_STATUS'] = 'UNKNOWN'



  #'AMERICAN INDIAN/ALASKA NATIVE'
  #'PORTUGUESE',
  #'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
  #'MIDDLE EASTERN'
  #'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE'
def getYear(series):
  return pd.to_datetime(series).dt.year

for i in range(len(dfs)):
  dfs[i][0] = dfs[i][0].rename(columns={'SUBJECT_ID_x':'SUBJECT_ID'})
  dfs[i][0]['AGE'] = getYear(dfs[i][0].merge(pat[['SUBJECT_ID','GENDER','DOB']],on='SUBJECT_ID')['ADMITTIME']) - getYear(dfs[i][0].merge(pat[['SUBJECT_ID','GENDER','DOB']],on='SUBJECT_ID')['DOB'])
  dfs[i][0] = dfs[i][0].merge(pat[['SUBJECT_ID','GENDER']],on='SUBJECT_ID')
for i in range(len(dfs)):
  dfs[i][0]['Faixa etária'] = 0
  dfs[i][0].loc[dfs[i][0]['AGE'] < 5,'Faixa etária'] = 1 #Bebes
  dfs[i][0].loc[((dfs[i][0]['AGE'] > 5) & (dfs[i][0]['AGE'] < 15) ),'Faixa etária'] = 2 #Crianças
  dfs[i][0].loc[((dfs[i][0]['AGE'] > 14) & (dfs[i][0]['AGE'] < 25) ),'Faixa etária'] = 3 #Adultos
  dfs[i][0].loc[((dfs[i][0]['AGE'] > 25) & (dfs[i][0]['AGE'] < 66) ),'Faixa etária'] = 4 #Adultos
  dfs[i][0].loc[dfs[i][0]['AGE'] > 66,'Faixa etária'] = 5 #Idosos
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Subset =['GENDER','Reduced_ETHNICITY','MARITAL_STATUS','AGE']
for i in range(len(dfs)):
  for j in Subset:
    dfs[i][0]['Coded '+ str(j)] = label_encoder.fit_transform(dfs[i][0][j])



500
1000
1500
2000
2500
3000






1








In [6]:
df = dfs[0][0]
df['CONGESTIVE HEART FAILURE'] = pd.get_dummies(df['DIAGNOSIS'])['CONGESTIVE HEART FAILURE']
df['CORONARY ARTERY DISEASE'] = pd.get_dummies(df['DIAGNOSIS'])['CORONARY ARTERY DISEASE']
df['DIABETIC KETOACIDOSIS'] = pd.get_dummies(df['DIAGNOSIS'])['DIABETIC KETOACIDOSIS']
df


,50912,50802,50931,50971,50960,51221,50868,51006,51250,51222,...,AGE,GENDER,Faixa etária,Coded GENDER,Coded Reduced_ETHNICITY,Coded MARITAL_STATUS,Coded AGE,CONGESTIVE HEART FAILURE,CORONARY ARTERY DISEASE,DIABETIC KETOACIDOSIS
0,-0.515011,-2.109932e-01,-0.335028,-0.956321,-0.415902,-1.267249,-0.898002,-0.540395,0.005088,-0.899363,...,71,M,5,1,4,1,55,False,True,False
1,-0.293598,-1.289350e-01,-0.365367,0.049241,-0.415902,0.060096,-0.318366,-0.447011,0.187060,0.275603,...,74,M,5,1,4,1,58,False,True,False
2,-0.274674,-4.359450e-02,-0.920991,0.000897,1.162108,0.413223,-1.245784,-0.589310,-1.187841,0.882284,...,70,M,5,1,4,1,54,False,True,False
3,-0.354912,1.471199e+00,1.064175,-0.724268,0.175852,-0.274757,-1.315340,0.118631,-0.571157,-0.254284,...,69,M,5,1,4,1,53,True,False,False
4,-0.611525,1.184337e-17,-0.562566,0.798579,-0.077757,1.175223,-0.318366,-0.749398,2.079570,0.759412,...,61,M,4,1,3,1,45,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3465,-0.361725,3.404377e-01,0.465904,0.145930,-0.711779,-0.844671,-0.898002,-0.393648,0.096074,-0.565304,...,67,M,5,1,4,1,51,False,True,False
3466,-0.491167,-9.337645e-02,-0.859881,-0.917645,-0.120025,-0.024438,-0.984947,-0.905927,-0.249673,0.372365,...,51,M,4,1,4,1,35,False,True,False
3467,0.052716,6.308112e-02,0.782939,1.969202,1.260733,-1.069588,-0.125154,0.611343,-0.702582,-0.650547,...,72,M,5,1,4,1,56,False,True,False
3468,-0.276566,-1.289350e-01,-1.132926,-0.086123,-0.284401,-0.276199,-1.129856,-0.567076,-0.267870,0.085535,...,82,M,5,1,2,3,66,False,True,False


In [7]:
df[df['CONGESTIVE HEART FAILURE']==1]

,50912,50802,50931,50971,50960,51221,50868,51006,51250,51222,...,AGE,GENDER,Faixa etária,Coded GENDER,Coded Reduced_ETHNICITY,Coded MARITAL_STATUS,Coded AGE,CONGESTIVE HEART FAILURE,CORONARY ARTERY DISEASE,DIABETIC KETOACIDOSIS
3,-0.354912,1.471199e+00,1.064175,-0.724268,0.175852,-0.274757,-1.315340,0.118631,-0.571157,-0.254284,...,69,M,5,1,4,1,53,True,False,False
4,-0.611525,1.184337e-17,-0.562566,0.798579,-0.077757,1.175223,-0.318366,-0.749398,2.079570,0.759412,...,61,M,4,1,3,1,45,True,False,False
13,1.440241,-1.156900e+00,0.464387,-0.257640,-0.004247,0.045868,2.892819,2.309157,-0.444490,-0.464341,...,87,M,5,1,4,1,71,True,False,False
14,-0.489016,2.949418e+00,-0.437396,-0.427456,0.157066,-0.979516,-1.761847,-0.206019,2.207578,-1.211575,...,75,F,5,0,4,1,59,True,False,False
16,0.115166,2.124270e-01,1.229823,0.677718,-0.866762,1.020941,0.527903,0.769652,-0.540829,0.920682,...,54,M,4,1,4,5,38,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3443,0.402273,2.977675e-01,-0.780568,-0.434202,2.176543,-0.196519,-0.003706,0.818440,0.077877,-0.401730,...,75,F,5,0,4,1,59,True,False,False
3447,0.444447,1.184337e-17,-0.726393,0.290963,-0.152900,-0.041847,-0.028548,0.237807,0.405427,-0.291145,...,81,M,5,1,4,1,65,True,False,False
3452,0.035684,2.322796e+00,-0.058369,-1.436248,-0.098585,-1.069588,-0.278951,1.487910,1.251321,-1.343378,...,300,M,5,1,4,5,74,True,False,False
3460,-0.170117,1.184337e-17,-0.602006,-1.111023,0.004901,1.582034,0.840907,-0.333616,0.257049,1.519684,...,85,F,5,0,4,5,69,True,False,False


In [8]:
df[df['CORONARY ARTERY DISEASE']==1]

,50912,50802,50931,50971,50960,51221,50868,51006,51250,51222,...,AGE,GENDER,Faixa etária,Coded GENDER,Coded Reduced_ETHNICITY,Coded MARITAL_STATUS,Coded AGE,CONGESTIVE HEART FAILURE,CORONARY ARTERY DISEASE,DIABETIC KETOACIDOSIS
0,-0.515011,-0.210993,-0.335028,-0.956321,-0.415902,-1.267249,-0.898002,-0.540395,0.005088,-0.899363,...,71,M,5,1,4,1,55,False,True,False
1,-0.293598,-0.128935,-0.365367,0.049241,-0.415902,0.060096,-0.318366,-0.447011,0.187060,0.275603,...,74,M,5,1,4,1,58,False,True,False
2,-0.274674,-0.043594,-0.920991,0.000897,1.162108,0.413223,-1.245784,-0.589310,-1.187841,0.882284,...,70,M,5,1,4,1,54,False,True,False
5,-0.036572,0.169757,-0.666475,-0.275984,1.083207,-0.449620,-0.637166,0.319467,-0.154138,-0.441472,...,80,M,5,1,3,1,64,False,True,False
6,-0.418498,-0.295616,-0.736101,0.673323,-0.240567,-0.106461,-0.654555,-0.669354,0.327039,0.008356,...,77,M,5,1,4,5,61,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3464,-0.191407,-0.034112,-0.468517,0.774406,-0.317276,-0.402749,-1.245784,-0.212216,-1.632661,-0.484669,...,65,M,4,1,4,1,49,False,True,False
3465,-0.361725,0.340438,0.465904,0.145930,-0.711779,-0.844671,-0.898002,-0.393648,0.096074,-0.565304,...,67,M,5,1,4,1,51,False,True,False
3466,-0.491167,-0.093376,-0.859881,-0.917645,-0.120025,-0.024438,-0.984947,-0.905927,-0.249673,0.372365,...,51,M,4,1,4,1,35,False,True,False
3467,0.052716,0.063081,0.782939,1.969202,1.260733,-1.069588,-0.125154,0.611343,-0.702582,-0.650547,...,72,M,5,1,4,1,56,False,True,False


In [9]:
df[df['DIABETIC KETOACIDOSIS']==1]


,50912,50802,50931,50971,50960,51221,50868,51006,51250,51222,...,AGE,GENDER,Faixa etária,Coded GENDER,Coded Reduced_ETHNICITY,Coded MARITAL_STATUS,Coded AGE,CONGESTIVE HEART FAILURE,CORONARY ARTERY DISEASE,DIABETIC KETOACIDOSIS
18,-0.475271,-2.475799e+00,2.198222,-0.772612,-1.402158,1.061952,1.826288,-0.918379,0.460018,1.001317,...,22,F,3,0,4,3,6,False,False,True
47,0.090834,1.184337e-17,0.210195,-1.470152,0.147673,1.366610,0.443442,-0.103967,0.187060,1.398731,...,47,M,4,1,4,1,31,False,False,True
67,-0.321468,1.184337e-17,1.113489,-0.724268,-1.155594,1.844429,1.789402,-0.733227,0.823962,1.951656,...,32,M,4,1,4,3,16,False,False,True
82,-0.540058,-2.475799e+00,1.561653,-0.383014,-1.511742,0.593248,1.761505,-0.939828,-0.040405,0.880364,...,21,F,3,0,4,3,5,False,False,True
88,-0.373080,-2.689150e+00,1.335602,-0.015218,-1.047106,1.322083,0.493125,-0.737539,-0.904773,1.565761,...,42,F,4,0,4,1,26,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3448,0.553735,-4.182608e+00,3.665839,1.868197,0.471729,-1.431296,1.211874,0.910395,0.591442,-1.275659,...,38,F,4,0,1,3,22,False,False,True
3449,-0.171370,-2.750107e+00,0.777853,-0.707205,0.792262,0.190943,1.311435,-0.236700,0.823962,-0.023898,...,39,F,4,0,1,3,23,False,False,True
3453,0.174777,-3.329203e+00,1.602074,0.726062,-1.204907,0.651836,3.058015,-0.440340,1.521522,0.667258,...,47,M,4,1,1,3,31,False,False,True
3454,-0.065909,-3.435879e+00,0.959883,1.428327,-0.556795,0.402837,2.968512,-0.795271,2.347979,0.128732,...,47,M,4,1,1,3,31,False,False,True


In [12]:
reffDf = {'Attr':[],
          'source':[],
          'Coded':[]}
protectedAttrs = ['GENDER','Reduced_ETHNICITY','MARITAL_STATUS','INSURANCE']

for i in protectedAttrs:
  temp = df[[i,'Coded '+ i ]].drop_duplicates().reset_index().drop(columns='index')
  for j in range(temp.shape[0]):
    reffDf['Attr'].append('Coded ' + i)
    reffDf['source'].append(temp.loc[j,i])
    reffDf['Coded'].append(temp.loc[j,'Coded '+i])
reffDf = pd.DataFrame(reffDf)

# Gradient boosting

## Insuficiencia Cardiaca

In [14]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'CONGESTIVE HEART FAILURE'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']:
  title = '# Atributo sensível: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test = train_test_split(df[examsList + CodedProtectedAttrs], df[disease], test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  # cr = CorrelationRemover(sensitive_feature_ids=CodedProtectedAttrs)
  # newDf = cr.fit_transform(X_resampled[examsList + CodedProtectedAttrs])
  # noCorrelationDF = pd.DataFrame(newDf, columns=[examsList])

  model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList+ CodedProtectedAttrs])
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)

pd.DataFrame(grupos)

# Atributo sensível: Coded GENDER

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.97      0.88      0.92       512
        True       0.73      0.92      0.81       182

    accuracy                           0.89       694
   macro avg       0.85      0.90      0.87       694
weighted avg       0.90      0.89      0.89       694



# Atributo sensível: Coded Reduced_ETHNICITY

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.96      0.88      0.92       512
        True       0.73      0.91      0.81       182

    accuracy                           0.89       694
   macro avg       0.85      0.89      0.86       694
weighted avg       0.90      0.89      0.89       694



# Atributo sensível: Coded INSURANCE

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.97      0.88      0.92       512
        True       0.73      0.93      0.81       182

    accuracy                           0.89       694
   macro avg       0.85      0.90      0.87       694
weighted avg       0.91      0.89      0.89       694



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


# Atributo sensível: Coded MARITAL_STATUS

Acurácia: 0.88

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.97      0.87      0.91       512
        True       0.71      0.91      0.80       182

    accuracy                           0.88       694
   macro avg       0.84      0.89      0.86       694
weighted avg       0.90      0.88      0.88       694



,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.908705,0.924627,0.908705,0.912629,0.930000,0.097035
1,F,0.843049,0.860239,0.843049,0.845528,0.902439,0.191489
2,OTHER,0.861789,0.920585,0.861789,0.880484,0.846154,0.136364
3,WHITE,0.895323,0.905574,0.895323,0.897884,0.902439,0.107362
4,BLACK,0.850000,0.859091,0.850000,0.849719,0.921053,0.214286
5,LATINO,0.962963,0.968254,0.962963,0.963936,1.000000,0.047619
6,ASIAN,0.933333,0.955556,0.933333,0.938667,1.000000,0.076923
7,Medicare,0.863636,0.886259,0.863636,0.866150,0.951049,0.185771
8,Private,0.909910,0.925601,0.909910,0.915921,0.727273,0.070000
9,Government,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [15]:
pd.set_option('display.float_format', '{:.4f}'.format)
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.908705 & 0.924627 & 0.908705 & 0.912629 & 0.930000 & 0.097035 \\
F & 0.843049 & 0.860239 & 0.843049 & 0.845528 & 0.902439 & 0.191489 \\
OTHER & 0.861789 & 0.920585 & 0.861789 & 0.880484 & 0.846154 & 0.136364 \\
WHITE & 0.895323 & 0.905574 & 0.895323 & 0.897884 & 0.902439 & 0.107362 \\
BLACK & 0.850000 & 0.859091 & 0.850000 & 0.849719 & 0.921053 & 0.214286 \\
LATINO & 0.962963 & 0.968254 & 0.962963 & 0.963936 & 1.000000 & 0.047619 \\
ASIAN & 0.933333 & 0.955556 & 0.933333 & 0.938667 & 1.000000 & 0.076923 \\
Medicare & 0.863636 & 0.886259 & 0.863636 & 0.866150 & 0.951049 & 0.185771 \\
Private & 0.909910 & 0.925601 & 0.909910 & 0.915921 & 0.727273 & 0.070000 \\
Government & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicaid & 0.946429 & 0.954887 & 0.946429 & 0.947681 & 1.000000 & 0.075000 \\
MARRIED & 0.878788 & 0.908474 & 0.878

In [16]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 93 & 335 & 36 & 7 \\
F & 74 & 114 & 27 & 8 \\
OTHER & 11 & 95 & 15 & 2 \\
WHITE & 111 & 291 & 35 & 12 \\
BLACK & 35 & 33 & 9 & 3 \\
LATINO & 6 & 20 & 1 & 0 \\
ASIAN & 2 & 12 & 1 & 0 \\
Medicare & 136 & 206 & 47 & 7 \\
Private & 16 & 186 & 14 & 6 \\
Government & 1 & 16 & 0 & 0 \\
Medicaid & 16 & 37 & 3 & 0 \\
MARRIED & 59 & 260 & 37 & 7 \\
SINGLE & 37 & 107 & 13 & 5 \\
WIDOWED & 49 & 39 & 11 & 3 \\
UNKNOWN & 5 & 12 & 1 & 0 \\
DIVORCED & 8 & 22 & 6 & 1 \\
SEPARATED & 8 & 4 & 0 & 0 \\
\bottomrule
\end{tabular}



## Cetoacidose diabética

In [17]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'DIABETIC KETOACIDOSIS'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']:
  title = '# Atributo sensível: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test = train_test_split(df[examsList + CodedProtectedAttrs], df[disease], test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList+ CodedProtectedAttrs])
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)


pd.DataFrame(grupos)

# Atributo sensível: Coded GENDER

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.97      0.98       604
        True       0.84      0.94      0.89        90

    accuracy                           0.97       694
   macro avg       0.92      0.96      0.94       694
weighted avg       0.97      0.97      0.97       694



# Atributo sensível: Coded Reduced_ETHNICITY

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.97      0.98       604
        True       0.82      0.94      0.88        90

    accuracy                           0.97       694
   macro avg       0.90      0.96      0.93       694
weighted avg       0.97      0.97      0.97       694



# Atributo sensível: Coded INSURANCE

Acurácia: 0.98

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.98      0.99       604
        True       0.88      0.94      0.91        90

    accuracy                           0.98       694
   macro avg       0.93      0.96      0.95       694
weighted avg       0.98      0.98      0.98       694



# Atributo sensível: Coded MARITAL_STATUS

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.98      0.98       604
        True       0.87      0.94      0.90        90

    accuracy                           0.97       694
   macro avg       0.93      0.96      0.94       694
weighted avg       0.98      0.97      0.97       694



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-17-06a771907026>:23: RuntimeWarning: invalid value encountered in scalar divide
  return tp/(tp+fn)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-17-06a771907026>:23: RuntimeWarning: invalid value encountered in scalar divide
  return tp/(tp+fn)


,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.9788,0.9811,0.9788,0.9795,0.9459,0.0184
1,F,0.9507,0.9534,0.9507,0.9514,0.9434,0.0471
2,OTHER,0.9593,0.9627,0.9593,0.9608,0.6667,0.0256
3,WHITE,0.9733,0.9765,0.9733,0.9742,0.9556,0.0248
4,BLACK,0.9125,0.9215,0.9125,0.9134,0.9677,0.1224
5,LATINO,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
6,ASIAN,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
7,Medicare,0.9773,0.9798,0.9773,0.9781,0.9310,0.0191
8,Private,0.9640,0.9652,0.9640,0.9644,0.9032,0.0262
9,Government,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000


In [18]:
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.978769 & 0.981079 & 0.978769 & 0.979491 & 0.945946 & 0.018433 \\
F & 0.950673 & 0.953358 & 0.950673 & 0.951416 & 0.943396 & 0.047059 \\
OTHER & 0.959350 & 0.962694 & 0.959350 & 0.960826 & 0.666667 & 0.025641 \\
WHITE & 0.973274 & 0.976546 & 0.973274 & 0.974231 & 0.955556 & 0.024752 \\
BLACK & 0.912500 & 0.921496 & 0.912500 & 0.913413 & 0.967742 & 0.122449 \\
LATINO & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
ASIAN & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicare & 0.977273 & 0.979802 & 0.977273 & 0.978097 & 0.931034 & 0.019074 \\
Private & 0.963964 & 0.965186 & 0.963964 & 0.964432 & 0.903226 & 0.026178 \\
Government & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicaid & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Self Pay & 1.000000 & 1.000000 & 1.00

In [19]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 35 & 426 & 8 & 2 \\
F & 50 & 162 & 8 & 3 \\
OTHER & 4 & 114 & 3 & 2 \\
WHITE & 43 & 394 & 10 & 2 \\
BLACK & 30 & 43 & 6 & 1 \\
LATINO & 6 & 21 & 0 & 0 \\
ASIAN & 2 & 13 & 0 & 0 \\
Medicare & 27 & 360 & 7 & 2 \\
Private & 28 & 186 & 5 & 3 \\
Government & 9 & 8 & 0 & 0 \\
Medicaid & 19 & 37 & 0 & 0 \\
Self Pay & 2 & 1 & 0 & 0 \\
MARRIED & 20 & 335 & 7 & 1 \\
SINGLE & 55 & 101 & 2 & 4 \\
WIDOWED & 2 & 99 & 1 & 0 \\
UNKNOWN & 0 & 17 & 1 & 0 \\
DIVORCED & 8 & 29 & 0 & 0 \\
SEPARATED & 0 & 10 & 2 & 0 \\
\bottomrule
\end{tabular}



## CORONARY ARTERY DISEASE

In [20]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'CORONARY ARTERY DISEASE'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']:
  title = '# Atributo sensível: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test,p_train,p_test = train_test_split(df[examsList], df[disease],df[CodedProtectedAttrs],
                                                                     test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList])
  x_test[protAttr] = p_test
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)

pd.DataFrame(grupos)

# Atributo sensível: Coded GENDER

Acurácia: 0.91

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.87      0.91      0.89       272
        True       0.94      0.91      0.93       422

    accuracy                           0.91       694
   macro avg       0.91      0.91      0.91       694
weighted avg       0.91      0.91      0.91       694



# Atributo sensível: Coded Reduced_ETHNICITY

Acurácia: 0.91

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.87      0.91      0.89       272
        True       0.94      0.91      0.93       422

    accuracy                           0.91       694
   macro avg       0.91      0.91      0.91       694
weighted avg       0.91      0.91      0.91       694



# Atributo sensível: Coded INSURANCE

Acurácia: 0.91

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.87      0.91      0.89       272
        True       0.94      0.91      0.93       422

    accuracy                           0.91       694
   macro avg       0.91      0.91      0.91       694
weighted avg       0.91      0.91      0.91       694



# Atributo sensível: Coded MARITAL_STATUS

Acurácia: 0.91

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.87      0.91      0.89       272
        True       0.94      0.91      0.93       422

    accuracy                           0.91       694
   macro avg       0.91      0.91      0.91       694
weighted avg       0.91      0.91      0.91       694



,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.9278,0.9316,0.9278,0.9288,0.9281,0.0730
1,F,0.8789,0.8799,0.8789,0.8793,0.8636,0.1111
2,OTHER,0.9024,0.9252,0.9024,0.9090,0.9038,0.1053
3,WHITE,0.9087,0.9088,0.9087,0.9087,0.9253,0.1190
4,BLACK,0.9125,0.9095,0.9125,0.9107,0.6364,0.0435
5,LATINO,0.9630,0.9658,0.9630,0.9631,0.9333,0.0000
6,ASIAN,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
7,Medicare,0.8939,0.8957,0.8939,0.8942,0.8839,0.0930
8,Private,0.9324,0.9341,0.9324,0.9331,0.9467,0.1132
9,Government,0.9412,0.9485,0.9412,0.9416,1.0000,0.1000


In [21]:
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.927813 & 0.931609 & 0.927813 & 0.928795 & 0.928144 & 0.072993 \\
F & 0.878924 & 0.879918 & 0.878924 & 0.879258 & 0.863636 & 0.111111 \\
OTHER & 0.902439 & 0.925173 & 0.902439 & 0.908971 & 0.903846 & 0.105263 \\
WHITE & 0.908686 & 0.908804 & 0.908686 & 0.908740 & 0.925267 & 0.119048 \\
BLACK & 0.912500 & 0.909464 & 0.912500 & 0.910731 & 0.636364 & 0.043478 \\
LATINO & 0.962963 & 0.965812 & 0.962963 & 0.963065 & 0.933333 & 0.000000 \\
ASIAN & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicare & 0.893939 & 0.895659 & 0.893939 & 0.894227 & 0.883929 & 0.093023 \\
Private & 0.932432 & 0.934116 & 0.932432 & 0.933060 & 0.946746 & 0.113208 \\
Government & 0.941176 & 0.948529 & 0.941176 & 0.941589 & 1.000000 & 0.100000 \\
Medicaid & 0.946429 & 0.947527 & 0.946429 & 0.946663 & 0.952381 & 0.057143 \\
Self Pay & 1.000000 & 1.000000 & 1.00

In [22]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 310 & 127 & 10 & 24 \\
F & 76 & 120 & 15 & 12 \\
OTHER & 94 & 17 & 2 & 10 \\
WHITE & 260 & 148 & 20 & 21 \\
BLACK & 7 & 66 & 3 & 4 \\
LATINO & 14 & 12 & 0 & 1 \\
ASIAN & 11 & 4 & 0 & 0 \\
Medicare & 198 & 156 & 16 & 26 \\
Private & 160 & 47 & 6 & 9 \\
Government & 7 & 9 & 1 & 0 \\
Medicaid & 20 & 33 & 2 & 1 \\
Self Pay & 1 & 2 & 0 & 0 \\
MARRIED & 255 & 73 & 14 & 21 \\
SINGLE & 54 & 98 & 3 & 7 \\
WIDOWED & 45 & 48 & 6 & 3 \\
UNKNOWN & 9 & 4 & 1 & 4 \\
DIVORCED & 19 & 17 & 0 & 1 \\
SEPARATED & 4 & 7 & 1 & 0 \\
\bottomrule
\end{tabular}



# SVM

## Insuficiencia Cardiaca

In [23]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'CONGESTIVE HEART FAILURE'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']:
  title = '# Atributo sensível: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test = train_test_split(df[examsList + CodedProtectedAttrs], df[disease], test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  # cr = CorrelationRemover(sensitive_feature_ids=CodedProtectedAttrs)
  # newDf = cr.fit_transform(X_resampled[examsList + CodedProtectedAttrs])
  # noCorrelationDF = pd.DataFrame(newDf, columns=[examsList])

  model = SVC(kernel='linear')
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList+ CodedProtectedAttrs])
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)

pd.DataFrame(grupos)

# Atributo sensível: Coded GENDER

Acurácia: 0.88

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.96      0.87      0.92       512
        True       0.72      0.90      0.80       182

    accuracy                           0.88       694
   macro avg       0.84      0.89      0.86       694
weighted avg       0.90      0.88      0.88       694



# Atributo sensível: Coded Reduced_ETHNICITY

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.97      0.88      0.92       512
        True       0.72      0.92      0.81       182

    accuracy                           0.89       694
   macro avg       0.85      0.90      0.87       694
weighted avg       0.91      0.89      0.89       694



# Atributo sensível: Coded INSURANCE

Acurácia: 0.88

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.97      0.87      0.92       512
        True       0.71      0.93      0.81       182

    accuracy                           0.88       694
   macro avg       0.84      0.90      0.86       694
weighted avg       0.90      0.88      0.89       694



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


# Atributo sensível: Coded MARITAL_STATUS

Acurácia: 0.88

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.97      0.86      0.91       512
        True       0.70      0.91      0.79       182

    accuracy                           0.88       694
   macro avg       0.83      0.89      0.85       694
weighted avg       0.90      0.88      0.88       694



,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.8960,0.9084,0.8960,0.8997,0.8700,0.0970
1,F,0.8475,0.8724,0.8475,0.8502,0.9390,0.2057
2,OTHER,0.8618,0.9302,0.8618,0.8820,0.9231,0.1455
3,WHITE,0.8976,0.9096,0.8976,0.9003,0.9187,0.1104
4,BLACK,0.8250,0.8401,0.8250,0.8241,0.9211,0.2619
5,LATINO,0.9630,0.9683,0.9630,0.9639,1.0000,0.0476
6,ASIAN,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
7,Medicare,0.8460,0.8717,0.8460,0.8489,0.9371,0.2055
8,Private,0.9189,0.9419,0.9189,0.9264,0.8636,0.0750
9,Government,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000


In [24]:
pd.set_option('display.float_format', '{:.4f}'.format)
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.895966 & 0.908434 & 0.895966 & 0.899667 & 0.870000 & 0.097035 \\
F & 0.847534 & 0.872378 & 0.847534 & 0.850174 & 0.939024 & 0.205674 \\
OTHER & 0.861789 & 0.930191 & 0.861789 & 0.882015 & 0.923077 & 0.145455 \\
WHITE & 0.897550 & 0.909611 & 0.897550 & 0.900319 & 0.918699 & 0.110429 \\
BLACK & 0.825000 & 0.840090 & 0.825000 & 0.824123 & 0.921053 & 0.261905 \\
LATINO & 0.962963 & 0.968254 & 0.962963 & 0.963936 & 1.000000 & 0.047619 \\
ASIAN & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicare & 0.845960 & 0.871663 & 0.845960 & 0.848873 & 0.937063 & 0.205534 \\
Private & 0.918919 & 0.941904 & 0.918919 & 0.926352 & 0.863636 & 0.075000 \\
Government & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicaid & 0.964286 & 0.968254 & 0.964286 & 0.964878 & 1.000000 & 0.050000 \\
MARRIED & 0.892562 & 0.915008 & 0.892

In [25]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 87 & 335 & 36 & 13 \\
F & 77 & 112 & 29 & 5 \\
OTHER & 12 & 94 & 16 & 1 \\
WHITE & 113 & 290 & 36 & 10 \\
BLACK & 35 & 31 & 11 & 3 \\
LATINO & 6 & 20 & 1 & 0 \\
ASIAN & 2 & 13 & 0 & 0 \\
Medicare & 134 & 201 & 52 & 9 \\
Private & 19 & 185 & 15 & 3 \\
Government & 1 & 16 & 0 & 0 \\
Medicaid & 16 & 38 & 2 & 0 \\
MARRIED & 59 & 265 & 32 & 7 \\
SINGLE & 39 & 103 & 17 & 3 \\
WIDOWED & 48 & 38 & 12 & 4 \\
UNKNOWN & 4 & 11 & 2 & 1 \\
DIVORCED & 9 & 21 & 7 & 0 \\
SEPARATED & 7 & 4 & 0 & 1 \\
\bottomrule
\end{tabular}



## Cetoacidose diabética

In [26]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'DIABETIC KETOACIDOSIS'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']:
  title = '# Atributo sensível: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test = train_test_split(df[examsList + CodedProtectedAttrs], df[disease], test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  model = SVC(kernel='linear')
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList+ CodedProtectedAttrs])
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)

pd.DataFrame(grupos)

# Atributo sensível: Coded GENDER

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       1.00      0.97      0.98       604
        True       0.85      0.98      0.91        90

    accuracy                           0.97       694
   macro avg       0.92      0.98      0.95       694
weighted avg       0.98      0.97      0.97       694



# Atributo sensível: Coded Reduced_ETHNICITY

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.97      0.98       604
        True       0.83      0.97      0.89        90

    accuracy                           0.97       694
   macro avg       0.91      0.97      0.94       694
weighted avg       0.97      0.97      0.97       694



# Atributo sensível: Coded INSURANCE

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.97      0.98       604
        True       0.84      0.97      0.90        90

    accuracy                           0.97       694
   macro avg       0.92      0.97      0.94       694
weighted avg       0.98      0.97      0.97       694



# Atributo sensível: Coded MARITAL_STATUS

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       1.00      0.97      0.98       604
        True       0.85      0.98      0.91        90

    accuracy                           0.97       694
   macro avg       0.92      0.98      0.95       694
weighted avg       0.98      0.97      0.97       694



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-26-15b79e267952>:23: RuntimeWarning: invalid value encountered in scalar divide
  return tp/(tp+fn)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-26-15b79e267952>:23: RuntimeWarning: invalid value encountered in scalar divide
  return tp/(tp+fn)


,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.9851,0.9875,0.9851,0.9857,1.0000,0.0161
1,F,0.9507,0.9550,0.9507,0.9517,0.9623,0.0529
2,OTHER,0.9756,0.9779,0.9756,0.9765,0.8333,0.0171
3,WHITE,0.9800,0.9823,0.9800,0.9806,0.9778,0.0198
4,BLACK,0.8875,0.9038,0.8875,0.8889,0.9677,0.1633
5,LATINO,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
6,ASIAN,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
7,Medicare,0.9697,0.9750,0.9697,0.9714,0.9310,0.0272
8,Private,0.9685,0.9721,0.9685,0.9694,0.9677,0.0314
9,Government,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000


In [27]:
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.985138 & 0.987502 & 0.985138 & 0.985720 & 1.000000 & 0.016129 \\
F & 0.950673 & 0.954996 & 0.950673 & 0.951682 & 0.962264 & 0.052941 \\
OTHER & 0.975610 & 0.977863 & 0.975610 & 0.976496 & 0.833333 & 0.017094 \\
WHITE & 0.979955 & 0.982315 & 0.979955 & 0.980591 & 0.977778 & 0.019802 \\
BLACK & 0.887500 & 0.903838 & 0.887500 & 0.888880 & 0.967742 & 0.163265 \\
LATINO & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
ASIAN & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicare & 0.969697 & 0.975044 & 0.969697 & 0.971367 & 0.931034 & 0.027248 \\
Private & 0.968468 & 0.972101 & 0.968468 & 0.969436 & 0.967742 & 0.031414 \\
Government & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicaid & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Self Pay & 1.000000 & 1.000000 & 1.00

In [28]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 37 & 427 & 7 & 0 \\
F & 51 & 161 & 9 & 2 \\
OTHER & 5 & 115 & 2 & 1 \\
WHITE & 44 & 396 & 8 & 1 \\
BLACK & 30 & 41 & 8 & 1 \\
LATINO & 6 & 21 & 0 & 0 \\
ASIAN & 2 & 13 & 0 & 0 \\
Medicare & 27 & 357 & 10 & 2 \\
Private & 30 & 185 & 6 & 1 \\
Government & 9 & 8 & 0 & 0 \\
Medicaid & 19 & 37 & 0 & 0 \\
Self Pay & 2 & 1 & 0 & 0 \\
MARRIED & 20 & 335 & 7 & 1 \\
SINGLE & 58 & 101 & 2 & 1 \\
WIDOWED & 2 & 98 & 2 & 0 \\
UNKNOWN & 0 & 16 & 2 & 0 \\
DIVORCED & 8 & 28 & 1 & 0 \\
SEPARATED & 0 & 10 & 2 & 0 \\
\bottomrule
\end{tabular}



## CORONARY ARTERY DISEASE

In [29]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'CORONARY ARTERY DISEASE'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']:
  title = '# Doenca atual: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','INSURANCE','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test = train_test_split(df[examsList + CodedProtectedAttrs], df[disease], test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  model = SVC(kernel='linear')
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList+ CodedProtectedAttrs])
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)

pd.DataFrame(grupos)

# Doenca atual: Coded GENDER

Acurácia: 0.90

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.85      0.90      0.87       272
        True       0.93      0.90      0.91       422

    accuracy                           0.90       694
   macro avg       0.89      0.90      0.89       694
weighted avg       0.90      0.90      0.90       694



# Doenca atual: Coded Reduced_ETHNICITY

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.84      0.88      0.86       272
        True       0.92      0.90      0.91       422

    accuracy                           0.89       694
   macro avg       0.88      0.89      0.88       694
weighted avg       0.89      0.89      0.89       694



# Doenca atual: Coded INSURANCE

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.84      0.89      0.86       272
        True       0.92      0.89      0.91       422

    accuracy                           0.89       694
   macro avg       0.88      0.89      0.89       694
weighted avg       0.89      0.89      0.89       694



# Doenca atual: Coded MARITAL_STATUS

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.84      0.89      0.87       272
        True       0.93      0.89      0.91       422

    accuracy                           0.89       694
   macro avg       0.88      0.89      0.89       694
weighted avg       0.89      0.89      0.89       694



,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.9087,0.9094,0.9087,0.9090,0.9311,0.1460
1,F,0.8744,0.8761,0.8744,0.8725,0.7727,0.0593
2,OTHER,0.9106,0.9434,0.9106,0.9181,0.8942,0.0000
3,WHITE,0.8820,0.8814,0.8820,0.8816,0.9146,0.1726
4,BLACK,0.8375,0.8182,0.8375,0.8264,0.2727,0.0725
5,LATINO,0.9630,0.9658,0.9630,0.9631,0.9333,0.0000
6,ASIAN,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
7,Medicare,0.8687,0.8711,0.8687,0.8691,0.8571,0.1163
8,Private,0.9279,0.9304,0.9279,0.9288,0.9408,0.1132
9,Government,0.8824,0.9085,0.8824,0.8832,1.0000,0.2000


In [30]:
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.908705 & 0.909366 & 0.908705 & 0.908993 & 0.931138 & 0.145985 \\
F & 0.874439 & 0.876096 & 0.874439 & 0.872517 & 0.772727 & 0.059259 \\
OTHER & 0.910569 & 0.943360 & 0.910569 & 0.918110 & 0.894231 & 0.000000 \\
WHITE & 0.881960 & 0.881449 & 0.881960 & 0.881589 & 0.914591 & 0.172619 \\
BLACK & 0.837500 & 0.818229 & 0.837500 & 0.826400 & 0.272727 & 0.072464 \\
LATINO & 0.962963 & 0.965812 & 0.962963 & 0.963065 & 0.933333 & 0.000000 \\
ASIAN & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
Medicare & 0.868687 & 0.871098 & 0.868687 & 0.869093 & 0.857143 & 0.116279 \\
Private & 0.927928 & 0.930434 & 0.927928 & 0.928807 & 0.940828 & 0.113208 \\
Government & 0.882353 & 0.908497 & 0.882353 & 0.883170 & 1.000000 & 0.200000 \\
Medicaid & 0.910714 & 0.910417 & 0.910714 & 0.910254 & 0.857143 & 0.057143 \\
Self Pay & 0.666667 & 0.833333 & 0.66

In [31]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 311 & 117 & 20 & 23 \\
F & 68 & 127 & 8 & 20 \\
OTHER & 93 & 19 & 0 & 11 \\
WHITE & 257 & 139 & 29 & 24 \\
BLACK & 3 & 64 & 5 & 8 \\
LATINO & 14 & 12 & 0 & 1 \\
ASIAN & 11 & 4 & 0 & 0 \\
Medicare & 192 & 152 & 20 & 32 \\
Private & 159 & 47 & 6 & 10 \\
Government & 7 & 8 & 2 & 0 \\
Medicaid & 18 & 33 & 2 & 3 \\
Self Pay & 1 & 1 & 1 & 0 \\
MARRIED & 251 & 74 & 13 & 25 \\
SINGLE & 53 & 93 & 8 & 8 \\
WIDOWED & 40 & 48 & 6 & 8 \\
UNKNOWN & 9 & 5 & 0 & 4 \\
DIVORCED & 19 & 16 & 1 & 1 \\
SEPARATED & 4 & 7 & 1 & 0 \\
\bottomrule
\end{tabular}



# Rede MLP

## Insuficiencia Cardiaca

In [32]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'CONGESTIVE HEART FAILURE'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded MARITAL_STATUS']:
  title = '# Doenca atual: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test = train_test_split(df[examsList + CodedProtectedAttrs], df[disease], test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  # cr = CorrelationRemover(sensitive_feature_ids=CodedProtectedAttrs)
  # newDf = cr.fit_transform(X_resampled[examsList + CodedProtectedAttrs])
  # noCorrelationDF = pd.DataFrame(newDf, columns=[examsList])

  model = MLPClassifier(hidden_layer_sizes=(20,), max_iter=500, random_state=42)
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList+ CodedProtectedAttrs])
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)

pd.DataFrame(grupos)

# Doenca atual: Coded GENDER

Acurácia: 0.86

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.94      0.88      0.91       512
        True       0.70      0.84      0.76       182

    accuracy                           0.86       694
   macro avg       0.82      0.86      0.83       694
weighted avg       0.88      0.86      0.87       694



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


# Doenca atual: Coded Reduced_ETHNICITY

Acurácia: 0.87

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.93      0.88      0.91       512
        True       0.71      0.82      0.76       182

    accuracy                           0.87       694
   macro avg       0.82      0.85      0.83       694
weighted avg       0.88      0.87      0.87       694



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


# Doenca atual: Coded MARITAL_STATUS

Acurácia: 0.86

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.94      0.87      0.90       512
        True       0.70      0.86      0.77       182

    accuracy                           0.86       694
   macro avg       0.82      0.86      0.84       694
weighted avg       0.88      0.86      0.87       694



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.8790,0.8884,0.8790,0.8823,0.8000,0.0997
1,F,0.8341,0.8487,0.8341,0.8366,0.8780,0.1915
2,OTHER,0.8537,0.8999,0.8537,0.8705,0.6923,0.1273
3,WHITE,0.8797,0.8876,0.8797,0.8821,0.8537,0.1104
4,BLACK,0.8000,0.8003,0.8000,0.7996,0.7632,0.1667
5,LATINO,0.8519,0.8757,0.8519,0.8587,0.8333,0.1429
6,ASIAN,0.9333,0.9556,0.9333,0.9387,1.0000,0.0769
7,MARRIED,0.8733,0.8918,0.8733,0.8794,0.8030,0.1111
8,SINGLE,0.8704,0.8878,0.8704,0.8748,0.8810,0.1333
9,WIDOWED,0.8529,0.8594,0.8529,0.8520,0.9231,0.2200


In [33]:
pd.set_option('display.float_format', '{:.4f}'.format)
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.878981 & 0.888356 & 0.878981 & 0.882302 & 0.800000 & 0.099730 \\
F & 0.834081 & 0.848724 & 0.834081 & 0.836551 & 0.878049 & 0.191489 \\
OTHER & 0.853659 & 0.899894 & 0.853659 & 0.870499 & 0.692308 & 0.127273 \\
WHITE & 0.879733 & 0.887625 & 0.879733 & 0.882125 & 0.853659 & 0.110429 \\
BLACK & 0.800000 & 0.800253 & 0.800000 & 0.799623 & 0.763158 & 0.166667 \\
LATINO & 0.851852 & 0.875731 & 0.851852 & 0.858730 & 0.833333 & 0.142857 \\
ASIAN & 0.933333 & 0.955556 & 0.933333 & 0.938667 & 1.000000 & 0.076923 \\
MARRIED & 0.873278 & 0.891834 & 0.873278 & 0.879407 & 0.803030 & 0.111111 \\
SINGLE & 0.870370 & 0.887754 & 0.870370 & 0.874762 & 0.880952 & 0.133333 \\
WIDOWED & 0.852941 & 0.859352 & 0.852941 & 0.852044 & 0.923077 & 0.220000 \\
UNKNOWN & 0.833333 & 0.847222 & 0.833333 & 0.837576 & 0.800000 & 0.153846 \\
DIVORCED & 0.810811 & 0.862850 & 0.810811 &

In [34]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 80 & 334 & 37 & 20 \\
F & 72 & 114 & 27 & 10 \\
OTHER & 9 & 96 & 14 & 4 \\
WHITE & 105 & 290 & 36 & 18 \\
BLACK & 29 & 35 & 7 & 9 \\
LATINO & 5 & 18 & 3 & 1 \\
ASIAN & 2 & 12 & 1 & 0 \\
MARRIED & 53 & 264 & 33 & 13 \\
SINGLE & 37 & 104 & 16 & 5 \\
WIDOWED & 48 & 39 & 11 & 4 \\
UNKNOWN & 4 & 11 & 2 & 1 \\
DIVORCED & 8 & 22 & 6 & 1 \\
SEPARATED & 6 & 4 & 0 & 2 \\
\bottomrule
\end{tabular}



## Cetoacidose diabética

In [35]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'DIABETIC KETOACIDOSIS'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded MARITAL_STATUS']:
  title = '# Doenca atual: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test = train_test_split(df[examsList + CodedProtectedAttrs], df[disease], test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  model = MLPClassifier(hidden_layer_sizes=(20,), max_iter=500, random_state=42)
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList+ CodedProtectedAttrs])
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)

pd.DataFrame(grupos)

# Doenca atual: Coded GENDER

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.98      0.98       604
        True       0.86      0.94      0.90        90

    accuracy                           0.97       694
   macro avg       0.93      0.96      0.94       694
weighted avg       0.97      0.97      0.97       694



# Doenca atual: Coded Reduced_ETHNICITY

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.97      0.98       604
        True       0.84      0.96      0.90        90

    accuracy                           0.97       694
   macro avg       0.92      0.96      0.94       694
weighted avg       0.97      0.97      0.97       694



# Doenca atual: Coded MARITAL_STATUS

Acurácia: 0.97

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.99      0.98      0.98       604
        True       0.85      0.96      0.90        90

    accuracy                           0.97       694
   macro avg       0.92      0.97      0.94       694
weighted avg       0.97      0.97      0.97       694



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-35-86481c7be5cc>:23: RuntimeWarning: invalid value encountered in scalar divide
  return tp/(tp+fn)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
<ipython-input-35-86481c7be5cc>:23: RuntimeWarning: invalid value encountered in scalar divide
  return tp/(tp+fn)


,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.9830,0.9842,0.9830,0.9834,0.9459,0.0138
1,F,0.9507,0.9534,0.9507,0.9514,0.9434,0.0471
2,OTHER,0.9756,0.9779,0.9756,0.9765,0.8333,0.0171
3,WHITE,0.9777,0.9798,0.9777,0.9783,0.9556,0.0198
4,BLACK,0.9125,0.9215,0.9125,0.9134,0.9677,0.1224
5,LATINO,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
6,ASIAN,1.0000,1.0000,1.0000,1.0000,1.0000,0.0000
7,MARRIED,0.9752,0.9788,0.9752,0.9764,0.9048,0.0205
8,SINGLE,0.9691,0.9693,0.9691,0.9692,0.9661,0.0291
9,WIDOWED,0.9804,0.9902,0.9804,0.9836,1.0000,0.0200


In [36]:
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.983015 & 0.984218 & 0.983015 & 0.983411 & 0.945946 & 0.013825 \\
F & 0.950673 & 0.953358 & 0.950673 & 0.951416 & 0.943396 & 0.047059 \\
OTHER & 0.975610 & 0.977863 & 0.975610 & 0.976496 & 0.833333 & 0.017094 \\
WHITE & 0.977728 & 0.979757 & 0.977728 & 0.978341 & 0.955556 & 0.019802 \\
BLACK & 0.912500 & 0.921496 & 0.912500 & 0.913413 & 0.967742 & 0.122449 \\
LATINO & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
ASIAN & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 1.000000 & 0.000000 \\
MARRIED & 0.975207 & 0.978833 & 0.975207 & 0.976434 & 0.904762 & 0.020468 \\
SINGLE & 0.969136 & 0.969323 & 0.969136 & 0.969190 & 0.966102 & 0.029126 \\
WIDOWED & 0.980392 & 0.990196 & 0.980392 & 0.983561 & 1.000000 & 0.020000 \\
UNKNOWN & 0.944444 & 1.000000 & 0.944444 & 0.971429 & NaN & 0.055556 \\
DIVORCED & 1.000000 & 1.000000 & 1.000000 & 1.00

In [37]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 35 & 428 & 6 & 2 \\
F & 50 & 162 & 8 & 3 \\
OTHER & 5 & 115 & 2 & 1 \\
WHITE & 43 & 396 & 8 & 2 \\
BLACK & 30 & 43 & 6 & 1 \\
LATINO & 6 & 21 & 0 & 0 \\
ASIAN & 2 & 13 & 0 & 0 \\
MARRIED & 19 & 335 & 7 & 2 \\
SINGLE & 57 & 100 & 3 & 2 \\
WIDOWED & 2 & 98 & 2 & 0 \\
UNKNOWN & 0 & 17 & 1 & 0 \\
DIVORCED & 8 & 29 & 0 & 0 \\
SEPARATED & 0 & 10 & 2 & 0 \\
\bottomrule
\end{tabular}



## CORONARY ARTERY DISEASE

In [38]:
from IPython.display import display, Markdown
from imblearn.combine import SMOTEENN
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score


def getProtectedClasses(attr,value):
  temp = reffDf[reffDf['Attr'] == attr]
  return temp[temp['Coded'] == value].reset_index()['source'][0]

def calculate_metrics(confusion_matrix, class_index):
    TP = confusion_matrix[class_index, class_index]
    FN = np.sum(confusion_matrix[class_index, :]) - TP
    FP = np.sum(confusion_matrix[:, class_index]) - TP
    TN = np.sum(confusion_matrix) - (FP + FN + TP)
    return TP, TN, FP, FN

def equal_Opportunity(confMatrix):
  tn, fp, fn, tp = confMatrix
  return tp/(tp+fn)
def equalized_odds(confMatrix):
  tn, fp, fn, tp = confMatrix
  return fp/(fp +tn)
def accuracy(confMatrix):
  tn, fp, fn, tp = confMatrix
  return (tp + tn) /  (tp + tn + fp + fn)

grupos = {'Classe':[],'Acurácia':[],'Precisão':[],'Recall':[],'F1-Score':[],
          'Equal Opportunity':[],'Equalized Odds':[] }


matrix = {'Classe':[],'TP':[],'TN':[],'FP':[],'FN':[]}

examsList = ['50912', '50802', '50931', '50971', '50960', '51221', '50868', '51006',
        '51250', '51222', '51277', '50983', '50882', '51301', '51249', '51248',
        '51279', '50804', '50970', '50902', '50820', '51237', '51265', '50893']

Subset =['GENDER','Reduced_ETHNICITY','MARITAL_STATUS','DIAGNOSIS']

for j in Subset:
  df['Coded '+ str(j)] = label_encoder.fit_transform(df[j])

# CodedProtectedAttrs = ['Coded GENDER','Coded Reduced_ETHNICITY','Coded INSURANCE','Coded MARITAL_STATUS']
disease = 'CORONARY ARTERY DISEASE'
for protAttr in ['Coded GENDER','Coded Reduced_ETHNICITY','Coded MARITAL_STATUS']:
  title = '# Doenca atual: ' + protAttr
  display(Markdown(title))

  CodedProtectedAttrs = [protAttr]
  protectedAttrs = ['GENDER','Reduced_ETHNICITY','MARITAL_STATUS']
  # diagnosis = ['CONGESTIVE HEART FAILURE','DIABETIC KETOACIDOSIS','CORONARY ARTERY DISEASE']

  x_train, x_test, y_train, y_test = train_test_split(df[examsList + CodedProtectedAttrs], df[disease], test_size=0.2, random_state=42)



  smote_enn = SMOTEENN(random_state=42)
  X_resampled, y_resampled = smote_enn.fit_resample(x_train,y_train)

  model = MLPClassifier(hidden_layer_sizes=(20,), max_iter=500, random_state=42)
  model.fit(X_resampled, y_resampled)

  y_pred = model.predict(x_test[examsList+ CodedProtectedAttrs])
  x_test['true ' + disease] = y_test
  x_test['pred ' + disease] = y_pred
  accuracy = accuracy_score(y_test, y_pred)
  report = classification_report(y_test, y_pred)
  # Imprimir os resultados
  print(f'Acurácia: {accuracy:.2f}')
  print('\nRelatório de Classificação:')
  print(report)


  for i in x_test[protAttr].unique():
    y_true = x_test[x_test[protAttr] == i]['true '  + disease]
    y_pred = x_test[x_test[protAttr] == i]['pred '  + disease]
    cm = confusion_matrix( y_true,y_pred)
    if len(cm) == 1:
      continue
    TP, TN, FP, FN = calculate_metrics(cm, 1)

    grupos['Classe'].append(getProtectedClasses(protAttr,i))
    # grupos['Ocorrências'].append(y_true.shape[0])
    grupos['Acurácia'].append(accuracy_score(y_true, y_pred))
    grupos['Precisão'].append(precision_score(y_true, y_pred, average='weighted'))
    grupos['Recall'].append(recall_score(y_true, y_pred, average='weighted'))
    grupos['F1-Score'].append(f1_score(y_true, y_pred, average='weighted'))
    grupos['Equal Opportunity'].append(equal_Opportunity([TN, FP, FN, TP]))
    grupos['Equalized Odds'].append(equalized_odds([TN, FP, FN, TP]))


    matrix['Classe'].append(getProtectedClasses(protAttr,i))
    matrix['TP'].append(TP)
    matrix['TN'].append(TN)
    matrix['FP'].append(FP)
    matrix['FN'].append(FN)

pd.DataFrame(grupos)

# Doenca atual: Coded GENDER

Acurácia: 0.90

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.88      0.87      0.87       272
        True       0.92      0.92      0.92       422

    accuracy                           0.90       694
   macro avg       0.90      0.89      0.90       694
weighted avg       0.90      0.90      0.90       694



# Doenca atual: Coded Reduced_ETHNICITY

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.88      0.85      0.86       272
        True       0.91      0.92      0.91       422

    accuracy                           0.89       694
   macro avg       0.89      0.89      0.89       694
weighted avg       0.89      0.89      0.89       694



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


# Doenca atual: Coded MARITAL_STATUS

Acurácia: 0.89

Relatório de Classificação:
              precision    recall  f1-score   support

       False       0.86      0.87      0.87       272
        True       0.92      0.91      0.91       422

    accuracy                           0.89       694
   macro avg       0.89      0.89      0.89       694
weighted avg       0.90      0.89      0.89       694



,Classe,Acurácia,Precisão,Recall,F1-Score,Equal Opportunity,Equalized Odds
0,M,0.9193,0.9190,0.9193,0.9191,0.9461,0.1460
1,F,0.8610,0.8613,0.8610,0.8611,0.8295,0.1185
2,OTHER,0.9350,0.9481,0.9350,0.9384,0.9327,0.0526
3,WHITE,0.8842,0.8838,0.8842,0.8831,0.9324,0.1964
4,BLACK,0.8750,0.8750,0.8750,0.8750,0.5455,0.0725
5,LATINO,0.9259,0.9259,0.9259,0.9259,0.9333,0.0833
6,ASIAN,0.9333,0.9467,0.9333,0.9354,0.9091,0.0000
7,MARRIED,0.9063,0.9063,0.9063,0.9063,0.9384,0.1954
8,SINGLE,0.8827,0.8824,0.8827,0.8825,0.8361,0.0891
9,WIDOWED,0.8725,0.8725,0.8725,0.8725,0.8542,0.1111


In [39]:
print(pd.DataFrame(grupos).to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
Classe & Acurácia & Precisão & Recall & F1-Score & Equal Opportunity & Equalized Odds \\
\midrule
M & 0.919321 & 0.919007 & 0.919321 & 0.919144 & 0.946108 & 0.145985 \\
F & 0.860987 & 0.861291 & 0.860987 & 0.861121 & 0.829545 & 0.118519 \\
OTHER & 0.934959 & 0.948120 & 0.934959 & 0.938428 & 0.932692 & 0.052632 \\
WHITE & 0.884187 & 0.883828 & 0.884187 & 0.883077 & 0.932384 & 0.196429 \\
BLACK & 0.875000 & 0.875000 & 0.875000 & 0.875000 & 0.545455 & 0.072464 \\
LATINO & 0.925926 & 0.925926 & 0.925926 & 0.925926 & 0.933333 & 0.083333 \\
ASIAN & 0.933333 & 0.946667 & 0.933333 & 0.935450 & 0.909091 & 0.000000 \\
MARRIED & 0.906336 & 0.906336 & 0.906336 & 0.906336 & 0.938406 & 0.195402 \\
SINGLE & 0.882716 & 0.882395 & 0.882716 & 0.882520 & 0.836066 & 0.089109 \\
WIDOWED & 0.872549 & 0.872545 & 0.872549 & 0.872463 & 0.854167 & 0.111111 \\
UNKNOWN & 0.722222 & 0.788889 & 0.722222 & 0.736158 & 0.692308 & 0.200000 \\
DIVORCED & 0.972973 & 0.974260 & 0.972973 &

In [40]:
print(pd.DataFrame(matrix).to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
Classe & TP & TN & FP & FN \\
\midrule
M & 316 & 117 & 20 & 18 \\
F & 73 & 119 & 16 & 15 \\
OTHER & 97 & 18 & 1 & 7 \\
WHITE & 262 & 135 & 33 & 19 \\
BLACK & 6 & 64 & 5 & 5 \\
LATINO & 14 & 11 & 1 & 1 \\
ASIAN & 10 & 4 & 0 & 1 \\
MARRIED & 259 & 70 & 17 & 17 \\
SINGLE & 51 & 92 & 9 & 10 \\
WIDOWED & 41 & 48 & 6 & 7 \\
UNKNOWN & 9 & 4 & 1 & 4 \\
DIVORCED & 20 & 16 & 1 & 0 \\
SEPARATED & 4 & 7 & 1 & 0 \\
\bottomrule
\end{tabular}

